WORK DONE BY THOMAS DANIEL AND THEO CHICHERY IN DIA3 


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.sparse.linalg import svds

Extraction of the first dataset 

In [2]:
columns_name=['user_id','item_id','rating','timestamp']
df=pd.read_csv("ml-100k/u.data",sep="\t",names=columns_name)        
df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


Extraction of the second dataset.

In [3]:

movie_columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] + [f'genre_{i}' for i in range(19)]
movies = pd.read_csv("ml-100k/u.item", sep='|', header=None, names=movie_columns, encoding='latin-1')
copy_movies = pd.read_csv("ml-100k/u.item", sep='|', header=None, names=movie_columns, encoding='latin-1')
movies.drop(columns=['title', 'release_date', 'video_release_date', 'imdb_url'], inplace=True)
movies

,movie_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:

ratings_data = df
movies_data = movies
movie_ratings = ratings_data[ratings_data['item_id'].isin(movies_data['movie_id'])]
user_movie_matrix = movie_ratings.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
R = user_movie_matrix.values
train_indices, test_indices = train_test_split(movie_ratings.index, test_size=0.2, random_state=42)
train_data = movie_ratings.copy()
train_data.loc[test_indices, 'rating'] = 0
train_user_movie_matrix = train_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
R_train = train_user_movie_matrix.values

def evaluate_svd_performance(k_values):
    rmse_scores = []
    mae_scores = []

    for k in k_values:
        U, sigma, Vt = svds(R_train, k=k)
        sigma = np.diag(sigma)
        predicted_ratings = np.dot(np.dot(U, sigma), Vt)
        test_data = movie_ratings.loc[test_indices]
        actual_ratings = []
        predicted_ratings_list = []
        for idx, row in test_data.iterrows():
            user_id = row['user_id']
            item_id = row['item_id']
            actual_rating = row['rating']
            user_idx = user_movie_matrix.index.get_loc(user_id)
            item_idx = user_movie_matrix.columns.get_loc(item_id)
            predicted_rating = predicted_ratings[user_idx, item_idx]
            actual_ratings.append(actual_rating)
            predicted_ratings_list.append(predicted_rating)

        rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings_list))
        mae = mean_absolute_error(actual_ratings, predicted_ratings_list)
        rmse_scores.append(rmse)
        mae_scores.append(mae)

        print(f"k = {k}: RMSE = {rmse}, MAE = {mae}")
    
    return rmse_scores, mae_scores,predicted_ratings

k_values = [5,10,15,20,30,40,50,75, 100,150,200]
rmse_scores, mae_scores,predicted_ratings = evaluate_svd_performance(k_values)


k = 5: RMSE = 2.6896774783754376, MAE = 2.42953791770712
k = 10: RMSE = 2.6115099045121366, MAE = 2.345648132251661
k = 15: RMSE = 2.618475928756285, MAE = 2.34923143571798
k = 20: RMSE = 2.643225610387362, MAE = 2.3732850041232223
k = 30: RMSE = 2.7221836238596033, MAE = 2.4516976438579827
k = 40: RMSE = 2.802256630373205, MAE = 2.533657046141666
k = 50: RMSE = 2.878369833467813, MAE = 2.6111322035937543
k = 75: RMSE = 3.049331922312527, MAE = 2.786548328059625
k = 100: RMSE = 3.175999286287317, MAE = 2.9212044050522685
k = 150: RMSE = 3.3562172777457264, MAE = 3.1208933033775743
k = 200: RMSE = 3.4762625943779626, MAE = 3.257172110458512


We can see that the best K value is between 10 and 15. 

In [9]:
def recommend_top_movies_for_user(user_id, num_recommendations=5):

    if user_id not in user_movie_matrix.index:
        print(f"User ID {user_id} not found in the dataset.")
        return None
    
    user_idx = user_movie_matrix.index.get_loc(user_id)
    user_predicted_ratings = predicted_ratings[user_idx]
    user_actual_ratings = user_movie_matrix.loc[user_id]
    rated_movies = user_actual_ratings[user_actual_ratings > 0].index.tolist()
    movie_ids = user_movie_matrix.columns
    
    predicted_ratings_df = pd.DataFrame({
        'movie_id': movie_ids,
        'predicted_rating': user_predicted_ratings
    })
    predicted_ratings_df = predicted_ratings_df[~predicted_ratings_df['movie_id'].isin(rated_movies)]
    top_movies = predicted_ratings_df.sort_values(by='predicted_rating', ascending=False).head(num_recommendations)
    top_movies_with_titles = top_movies.merge(movies_data, on='movie_id')
    return top_movies_with_titles[['movie_id', 'predicted_rating']]

top_movies_for_user = recommend_top_movies_for_user(user_id=201, num_recommendations=5)
print(top_movies_for_user)


   movie_id  predicted_rating
0       504          1.601717
1       732          1.049295
2       343          1.019013
3       902          0.989835
4       287          0.965780


Function to extract the top 5 rated films by user 

We keep the movie id and title to then match the ouput of the precedent function 

In [10]:
title_movie_ids = copy_movies[['movie_id', 'title']]
title_movie_ids

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


We do the final merge to get the title of the top 5 films for the user 

In [11]:

predicted_ratings_with_titles = top_movies_for_user.merge(title_movie_ids, on='movie_id')
print(predicted_ratings_with_titles)

   movie_id  predicted_rating                       title
0       504          1.601717     Bonnie and Clyde (1967)
1       732          1.049295                 Dave (1993)
2       343          1.019013  Alien: Resurrection (1997)
3       902          0.989835    Big Lebowski, The (1998)
4       287          0.965780        Marvin's Room (1996)
